In [14]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [15]:
latent_dim = 100

# Generator
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()

    # block
    def block(input_dim, output_dim, normalize=True):
      layers = [nn.Linear(input_dim, output_dim)]
      if normalize :
        # 배치정규화
          layers.append(nn.BatchNorm1d(output_dim, 0.8))
      layers.append(nn.LeakyReLU(0.2, inplace=True))
      return layers

    #여러개의 블록
    self.model = nn.Sequential(
        *block(latent_dim, 128, normalize = False),
        *block(128, 256),
        *block(256, 512),
        *block(512, 1024),
        nn.Linear(1024, 1*28*28),
        nn.Tanh()
    )
    
    def forward(self, z):
      img = self.model(z)
      img = img.view(img.size(0),1,28,28)
      return img

In [16]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()

    self.model = nn.Sequential(
        nn.Linear(1*28,28, 512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(256, 1),
        nn.Sigmoid(),
    )

    def forward(self, img):
      flattened = img.view( img.size(0), -1)
      output = self.model(flattened)

      return output

In [17]:
transforms_train = transforms.Compose([
          transforms.Resize(28),
          transforms.ToTensor(),
          transforms.Normalize([0.5],[0.5])
])

train_dataset = datasets.MNIST(root = "./dataset", train = True, download  = True , transform = transforms_train)
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size =128, shuffle =True, num_workers = 4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [18]:
# 초기화
generator = Generator()
discriminator = Discriminator()

generator.cuda()
discriminator.cuda()

# loss
adversarial_loss = nn.BCELoss()
adversarial_loss.cuda()

# 학습률
lr = 0.0002

# 최적화 함수
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr , betas = (0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas = (0.5, 0.999))

In [21]:
import time
n_epochs = 200 
sample_interval = 2000
start_time = time.time()

for epoch in range(n_epochs) : 
    for i, (imgs, _) in enumerate(dataloader):

      real = torch.cuda.FloatTensor(imgs.size(0),1).fill_(1.0) # 진짜(real) : 1
      fake = torch.cuda.FloatTensor(imgs.size(0),1).fill_(0.0) # 가짜(fake) : 0

      real_imgs = imgs.cuda()
      """ 생성자 학습 """
      optimizer_G.zero_grad()

      #랜덤 노이즈 샘플링
      z = torch.normal(mean=0, std =1 , size=(imgs.shape[0],latent_dim)).cuda()

      #이미지 생성
      generated_imgs = generator(z)

      #생성자(generator)의 손실(loss) 값 계산
      g_loss = adversarial_loss(discriminator(generated_imgs), real)

      # 생성자 업데이트
      g_loss.backward()
      optimizer_G.step()

      """ 판별자 학습 """
      optimizer_D.zero_grad()

      real_loss = adversarial_loss(discriminator(generated_imgs), real)
      fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake)
      loss_d = (real_loss+fake_loss)/2  

      d_loss.backward()
      optimizer_D.step()

      done = epoch *len(dataloader) + i 
      if done % sample_interval == 0 :
        save_image(generated_imgs.data[:25], f"{done}.png", nrow=5, normalize=True)

    print(f"[Epoch {epoch}/{n_epochs}] [D loss:{d_loss.item() :.6f}] [G loss: {g_loss.item():.6f}][Elapsed time: {time.time()-start_time:.2f}s]")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


NotImplementedError: ignored

In [ ]:
  im